In [13]:
#load the libraries we need
import json
import bz2
import zipfile
import pandas as pd
import orjson

#function for calculating relative frequencies 
def rel_freq(x):
    
    #calculate relative frequency for every calculated language for every page
    freqs = [(value, round(x.count(value) / len(x), 2)) for value in set(x)] 
    
    #create a dataframe out of these relative frequencies
    df = pd.DataFrame(freqs, columns = ['Value', 'Frequency'])
    
    #filter down to the language present in over 70% of the text
    df = df[df['Frequency'] > 0.70]
    
    #if the value column contains 'en'
    if df['Value'].str.contains("en").any(): 
        
        #append the name of the relevant path to the engtexts file
        with open("engtexts.txt", "a") as myfile:
            myfile.write(name[34:len(name)] + "\n")

#pull in the list of filenames + pathws
file_path = 'file_listing.txt'

#open it
with open(file_path, 'r') as file:
    #readlines
    path = file.read().splitlines()

#load all the ids we've already confirmed as being written in english
english_texts = 'engtexts.txt'

#open
with open(english_texts, 'r') as file:
    #read lines
    english_texts = file.read().splitlines()

#find the position of the last english_text in the list of paths
index = path.index(english_texts[-1])

#remove everything before this point from the list of paths
path = path[index:len(path)]

print(path[1])

aeu/a+30385/aeu.ark+=13960=t3pv82155.json.bz2


In [ ]:
#add the actual folder locations to the start of each id
prefix = '../../Volumes/My Passport for Mac/'

#add the prefix
paths = [prefix + filename for filename in path]

#create empty variable calculated languages
calculated_languages = []

#for every name in paths
for name in paths:
    
    #read in the zip file
    with bz2.BZ2File(name) as input_file:
        
        #read it
        input_file_content = input_file.read()
        
        #decode
        json_input = input_file_content.decode('utf-8')
        
        #and load the json
        data = orjson.loads(json_input)

        #extract the features from the json
        features = data.get('features', {})

        #check if pages is where we need it to be
        if 'pages' in features and isinstance(features['pages'], list):
          
            #loop over every page in features
            for page in features['pages']:
                
                #extract calculated language
                calculated_language = page.get('calculatedLanguage')
                if calculated_language:
                    
                    #append it to calculated languages
                    calculated_languages.append(calculated_language)

    rel_freq(calculated_languages)